# Part 3: Collaborative Filtering and Hybrid Approach

## 8. Collaborative Filtering Integration

In this section, we implement **Item-Based Collaborative Filtering** using **Centered Cosine Similarity (Pearson Correlation)**.

**Approach Formalization**:
1.  **Item-Item Similarity**: We compute the similarity between items based on their co-ratings by users. Two items are similar if the same users rated them similarly.
2.  **Normalization**: We subtract the user's mean rating from each rating to handle optimism/pessimism bias. This effectively turns Cosine Similarity into Pearson Correlation.
3.  **Prediction**: $\hat{r}_{ui} = \mu_u + \frac{\sum_{j \in N(i)} w_{ij} (r_{uj} - \mu_u)}{\sum_{j \in N(i)} |w_{ij}|}$
    *   Where $w_{ij}$ is the similarity between item $i$ and item $j$.
    *   $N(i)$ are the k-nearest neighbors of item $i$ that user $u$ has rated.
    *   $\mu_u$ is the average rating of user $u$.

In [ ]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix, diags
from scipy.sparse.linalg import norm
import os

RESULTS_DIR = "../results"
if not os.path.exists(RESULTS_DIR):
    os.makedirs(RESULTS_DIR)

### Subtask 2: Build the user–item rating matrix

We construct a sparse matrix where rows are Users and columns are Items. This is slightly counter-intuitive for "Item-based" similarity (which likes Items as rows for dot product), but standard datasets are $U \times I$. We can Transpose for similarity computation.

In [ ]:
def build_user_item_matrix(df_interactions):
    """
    Constructs sparse User-Item Rating Matrix (Rows=Users, Cols=Items).
    Returns: Matrix R, user_map (id->idx), item_map (id->idx)
    """
    print("\n--- Building User-Item Matrix ---")
    
    # Unique IDs
    users = sorted(df_interactions['user_id'].unique())
    items = sorted(df_interactions['item_id'].unique())
    
    user_map = {u: i for i, u in enumerate(users)}
    item_map = {it: i for i, it in enumerate(items)}
    
    # Map Data
    row_indices = df_interactions['user_id'].map(user_map)
    col_indices = df_interactions['item_id'].map(item_map)
    ratings = df_interactions['rating'].values
    
    # Build CSR
    # Shape: (n_users, n_items)
    R = csr_matrix((ratings, (row_indices, col_indices)), shape=(len(users), len(items)))
    
    print(f"Matrix Shape: {R.shape} with {R.nnz} ratings.")
    print(f"Sparsity: {1.0 - R.nnz / (R.shape[0] * R.shape[1]):.4%}")
    
    return R, user_map, item_map

### Subtask 3: Normalize ratings

We perform **Mean-Centering**. For each user, we calculate their average rating and subtract it from their ratings. This creates a matrix of deviations.
We perform this manually to keep sparsity structure.

In [ ]:
def center_ratings_by_user(R):
    """
    Subtracts row mean from non-zero elements.
    Returns: Centered Matrix R_centered, User Means vector
    """
    print("\n--- Mean-Centering Ratings ---")
    
    # Calculate Row Means (User Means)
    # Sum of ratings / Count of non-zero ratings
    row_sums = np.array(R.sum(axis=1)).flatten()
    row_counts = np.diff(R.indptr)
    
    # Avoid division by zero
    with np.errstate(divide='ignore', invalid='ignore'):
        user_means = row_sums / row_counts
    user_means[~np.isfinite(user_means)] = 0.0
    
    # Create Centered Matrix
    # To maintain sparsity, we only subtract from existing ratings.
    # Non-rated items remain 0 (conceptually undefined, but treated as neutral 0 in centered space? 
    # Actually Pearson is defined only on common items. In sparse matrices, 0 usually means missing.
    # If we subtract mean, 0 becomes -mean, making matrix dense. 
    # Optimization: We ONLY adjust specific non-zero entries. 
    # Similarity between i and j uses ONLY users who rated BOTH.
    
    R_coo = R.tocoo()
    rows = R_coo.row
    cols = R_coo.col
    data = R_coo.data
    
    # Subtract user mean from each rating
    new_data = data - user_means[rows]
    
    R_centered = csr_matrix((new_data, (rows, cols)), shape=R.shape)
    
    print("Mean Centering complete.")
    return R_centered, user_means

### Subtask 4: Compute item–item similarity

We calculate the Cosine Similarity between distinct **columns** of $R_{centered}$.
Since $R_{centered}$ has user biases removed, this is equivalent to Pearson Correlation.

In [ ]:
def compute_item_similarity_matrix(R_centered):
    """
    Computes Cosine Similarity between columns (Items).
    Sim(i, j) = (Ri . Rj) / (|Ri| |Rj|)
    """
    print("\n--- Computing Item-Item Similarity matrix ---")
    
    # Transpose so Items are Rows -> easier dot product
    # Matrix M: (n_items, n_users)
    M = R_centered.T.tocsr()
    
    # Compute Norms
    print("Calculating Item Norms...")
    # row_norms for M = column norms for R_centered
    item_norms = np.sqrt(np.array(M.power(2).sum(axis=1)).flatten())
    
    # Avoid division by zero
    item_norms[item_norms == 0] = 1e-9
    inv_norms = 1.0 / item_norms
    
    # Normalize rows of M (so dot product becomes cosine)
    M_normalized = diags(inv_norms) @ M
    
    # Compute Dot Product: (N_items, N_items)
    print("Computing Dot Product (Similarity)...")
    # This can be heavy if N_items is huge. 
    # For this assignment, assuming it fits in memory or we sparsify.
    sim_matrix = M_normalized.dot(M_normalized.T)
    
    # Set diagonal to 0 (ignore self-similarity)
    sim_matrix.setdiag(0.0)
    
    # Sparsify: drop near-zero values to save memory
    sim_matrix.eliminate_zeros()
    
    print(f"Similarity Matrix Shape: {sim_matrix.shape}, NNZ: {sim_matrix.nnz}")
    return sim_matrix

### Subtask 5: Select neighborhood size (k)

Function to extract local top-K similar items.

In [ ]:
def get_k_neighbors(sim_matrix, k=20):
    """
    Use the full similarity matrix. For prediction, we will select top-k on the fly.
    This function is a placeholder configuration.
    """
    print(f"Using Neighborhood size k={k}")
    return k

### Subtask 6 & 7: Predict ratings and Handle Edge Cases

We predict rating for user $u$ on item $i$.

In [ ]:
def predict_item_based_cf(user_idx, item_idx, R, sim_matrix, user_means, k=20):
    """
    Predicts rating: mu_u + [Sum(sim * (r - mu_u)) / Sum(|sim|)]
    Uses R (sparse) directly.
    """
    
    # 1. Get Similarity row for target item
    # Row 'item_idx' in sim_matrix contains similarities to all other items
    # Extract row efficiently
    sim_row = sim_matrix.getrow(item_idx)
    
    # 2. Get User's Rated items
    # Row 'user_idx' in R contains items user rated
    user_row = R.getrow(user_idx)
    
    # 3. Find intersection: Items j that are (Similar to i) AND (Rated by u)
    # We can iterate over user's rated items since that set is usually smaller
    rated_indices = user_row.indices
    rated_values = user_row.data
    
    if len(rated_indices) == 0:
         return user_means[user_idx] # Fallback: User Mean
    
    # Extract similarities for these items
    # dense conversion for fast indexing if items are not too many, or iterative check
    # For assignment scale, dense row is fine
    
    relevant_sims = []
    relevant_ratings = []
    
    # Optim: Iterate only Non-Zero similarities if Sim row is sparse
    # But we need intersection. 
    # Let's map rated items to their ratings
    rating_map = {idx: val for idx, val in zip(rated_indices, rated_values)}
    
    # Iterate Top-K neighbors from Sim Row
    # To do that, we get indices and data from sim_row
    neighbor_indices = sim_row.indices
    neighbor_scores = sim_row.data
    
    # Filter: Neighbors must have been rated by user
    candidates = []
    for j, score in zip(neighbor_indices, neighbor_scores):
        if j in rating_map:
            candidates.append((score, rating_map[j]))
            
    # Sort by absolute similarity (Top-K)
    candidates.sort(key=lambda x: abs(x[0]), reverse=True)
    top_k = candidates[:k]
    
    if not top_k:
        return user_means[user_idx] # No common neighbors
        
    weighted_sum = 0.0
    sum_abs_sim = 0.0
    
    mu_u = user_means[user_idx]
    
    for score, r_val in top_k:
        # Prediction formula uses deviation: (r - mu_u)
        # r_val is raw rating.
        dev = r_val - mu_u
        weighted_sum += score * dev
        sum_abs_sim += abs(score)
        
    if sum_abs_sim == 0:
        return mu_u
        
    pred = mu_u + (weighted_sum / sum_abs_sim)
    
    # Clip to valid range
    pred = max(1.0, min(5.0, pred))
    return pred

### Subtask 8, 9, 10: Generate Recommendations, Save, and Validate

Orchestrate the process for a set of target users.

In [ ]:
def run_collaborative_filtering_pipeline(df_interactions, df_items_map, target_user_ids=None):
    """
    Full execution wrapper.
    """
    print("\n==============================================")
    print("    STARTING ITEM-BASED COLLABORATIVE FILTERING    ")
    print("==============================================")
    
    # 1. Build Matrix
    R, user_map, item_map = build_user_item_matrix(df_interactions)
    inv_user_map = {v: k for k, v in user_map.items()}
    inv_item_map = {v: k for k, v in item_map.items()}
    
    # 2. Normalize
    R_centered, user_means = center_ratings_by_user(R)
    
    # 3. Similarity
    sim_matrix = compute_item_similarity_matrix(R_centered)
    
    # 4. Generate Recs for Targets
    if target_user_ids is None:
        # Pick top 5 active users
        target_user_ids = df_interactions['user_id'].value_counts().head(5).index.tolist()
        
    print(f"\nGenerating recommendations for {len(target_user_ids)} users...")
    
    all_recs = []
    
    for uid in target_user_ids:
        if uid not in user_map:
            print(f"User {uid} not in matrix. Skipping.")
            continue
            
        u_idx = user_map[uid]
        
        # Identify unrated items (Candidates)
        rated_indices = R.getrow(u_idx).indices
        rated_set = set(rated_indices)
        
        candidates = []
        preds = []
        
        # For demonstration, we can't score ALL items if N is huge. 
        # But if N ~ 1000-5000, we can.
        # Let's score all.
        
        N_items = R.shape[1]
        
        # Heuristic: Only score items that appear in Top-K neighbors of rated items?
        # Strict Item-based approach involves iterating all candidates.
        # We will iterate all columns.
        
        for i_idx in range(N_items):
            if i_idx not in rated_set:
                score = predict_item_based_cf(u_idx, i_idx, R, sim_matrix, user_means, k=20)
                preds.append((score, i_idx))
        
        # Top 10
        preds.sort(key=lambda x: x[0], reverse=True)
        top_10 = preds[:10]
        
        # Format
        rank = 1
        for score, i_idx in top_10:
            item_id = inv_item_map[i_idx]
            # Get title attempt
            title = "Unknown"
            items_row = df_items_map[df_items_map['item_id'] == item_id]
            if not items_row.empty:
                title = items_row.iloc[0]['title']
                
            all_recs.append({
                'User': uid,
                'Rank': rank,
                'Item_ID': item_id,
                'Predicted_Rating': round(score, 4),
                'Title': title
            })
            rank += 1
            
    # Save Results
    df_out = pd.DataFrame(all_recs)
    save_path = os.path.join(RESULTS_DIR, "cf_item_based_recommendations.csv")
    df_out.to_csv(save_path, index=False)
    print(f"\nRecommendations saved to {save_path}")
    
    # Validate: Print first few
    print(df_out.head(10))
    return df_out

In [ ]:
def run_collaborative_filtering_pipeline(df_interactions, df_items_map, target_user_ids=None):
    """
    Full execution wrapper. UPDATED to return matrices for SVD.
    """
    print("\n==============================================")
    print("    STARTING ITEM-BASED COLLABORATIVE FILTERING    ")
    print("==============================================")
    
    # 1. Build Matrix
    R, user_map, item_map = build_user_item_matrix(df_interactions)
    inv_user_map = {v: k for k, v in user_map.items()}
    inv_item_map = {v: k for k, v in item_map.items()}
    
    # 2. Normalize
    R_centered, user_means = center_ratings_by_user(R)
    
    # 3. Similarity
    sim_matrix = compute_item_similarity_matrix(R_centered)
    
    # 4. Generate Recs for Targets
    if target_user_ids is None:
        # Pick top 5 active users
        target_user_ids = df_interactions['user_id'].value_counts().head(5).index.tolist()
        
    print(f"\nGenerating recommendations for {len(target_user_ids)} users...")
    
    all_recs = []
    
    for uid in target_user_ids:
        if uid not in user_map:
            print(f"User {uid} not in matrix. Skipping.")
            continue
            
        u_idx = user_map[uid]
        
        # Identify unrated items (Candidates)
        rated_indices = R.getrow(u_idx).indices
        rated_set = set(rated_indices)
        
        candidates = []
        preds = []
        
        # For demonstration, we can't score ALL items if N is huge. 
        # But if N ~ 1000-5000, we can.
        # Let's score all.
        
        N_items = R.shape[1]
        
        # Heuristic: Only score items that appear in Top-K neighbors of rated items?
        # Strict Item-based approach involves iterating all candidates.
        # We will iterate all columns.
        
        for i_idx in range(N_items):
            if i_idx not in rated_set:
                score = predict_item_based_cf(u_idx, i_idx, R, sim_matrix, user_means, k=20)
                preds.append((score, i_idx))
        
        # Top 10
        preds.sort(key=lambda x: x[0], reverse=True)
        top_10 = preds[:10]
        
        # Format
        rank = 1
        for score, i_idx in top_10:
            item_id = inv_item_map[i_idx]
            # Get title attempt
            title = "Unknown"
            items_row = df_items_map[df_items_map['item_id'] == item_id]
            if not items_row.empty:
                title = items_row.iloc[0]['title']
                
            all_recs.append({
                'User': uid,
                'Rank': rank,
                'Item_ID': item_id,
                'Predicted_Rating': round(score, 4),
                'Title': title
            })
            rank += 1
            
    # Save Results
    df_out = pd.DataFrame(all_recs)
    save_path = os.path.join(RESULTS_DIR, "cf_item_based_recommendations.csv")
    df_out.to_csv(save_path, index=False)
    print(f"\nRecommendations saved to {save_path}")
    
    # Validate: Print first few
    print(df_out.head(10))
    
    # RETURN MATRICES FOR SVD
    return df_out, R, R_centered, user_means, user_map, item_map

## 8.2. Matrix Factorization (SVD)

We implement **Truncated SVD** to uncover latent factors.
We reuse the centered user-item matrix $R_{centered}$ from the previous step.
$$ R_{centered} \approx U \Sigma V^T $$
Prediction: $\hat{r}_{ui} = (U_u \cdot \Sigma \cdot V_i^T) + \mu_u$

In [ ]:
from scipy.sparse.linalg import svds

# Subtasks 1, 2, 3: Reuse and Prepare Matrix
# We already have R_centered from the CF pipeline.
# Missing values in R (sparse) are 0.
# Since R_centered = R - mu, the 0s represent 'average rating' (mean imputation implicitly),
# which is a common strategy for SVD to avoid treating missing as 'dislike' (0 stars).
def prepare_matrix_for_svd(R_centered):
    """
    Verifies the matrix is ready for SVD.
    """
    print("\n--- Preparing Matrix for SVD ---")
    # Conversion to float for SVD precision
    R_float = R_centered.asfptype()
    print(f"Matrix Properties: Shape={R_float.shape}, NNZ={R_float.nnz}, Dtype={R_float.dtype}")
    return R_float

In [ ]:
# Subtask 4: Apply Truncated SVD
def apply_truncated_svd(matrix, k=20):
    """
    Decomposes the matrix using Truncated SVD.
    Returns U, Sigma (diagonal), Vt.
    """
    print(f"\n--- Applying Truncated SVD (k={k}) ---")
    
    # svds returns: 
    # u: Unitary matrix having left singular vectors as columns.
    # s: The singular values.
    # vt: Unitary matrix having right singular vectors as rows.
    u, s, vt = svds(matrix, k=k)
    
    # svds returns them in ascending order of singular values, we usually want descending
    # so we flip them
    u = u[:, ::-1]
    s = s[::-1]
    vt = vt[::-1, :]
    
    # Convert Sigma to diagonal matrix
    sigma = np.diag(s)
    
    print(f"SVD Complete.")
    print(f"U shape: {u.shape}")
    print(f"Sigma shape: {sigma.shape}")
    print(f"Vt shape: {vt.shape}")
    
    return u, sigma, vt

In [ ]:
# Subtasks 5 & 6: Reconstruct and Clip
def reconstruct_predictions(u, sigma, vt, user_means):
    """
    Computes prediction matrix: P = (U * Sigma * Vt) + UserMeans
    """
    print("\n--- Reconstructing Prediction Matrix ---")
    
    # 1. Dot Product
    # U (N_users, k) . Sigma (k, k) . Vt (k, N_items)
    # Result is Dense (N_users, N_items)
    pred_centered = np.dot(np.dot(u, sigma), vt)
    
    # 2. Add Means back (Broadcasting)
    # user_means is (N_users, )
    # reshaped to (N_users, 1) to add to each column
    means_reshaped = user_means.reshape(-1, 1)
    predicted_ratings = pred_centered + means_reshaped
    
    # 3. Clip to Valid Range [1, 5]
    predicted_ratings = np.clip(predicted_ratings, 1.0, 5.0)
    
    print(f"Reconstruction Complete. Shape: {predicted_ratings.shape}")
    return predicted_ratings

In [ ]:
# Subtasks 7, 8, 9, 10: Generate Recommendations for Target Users
def generate_svd_recommendations(predicted_matrix, df_items_map, user_map, target_users, k_factors):
    """
    Extracts top N items for target users from the dense predicted matrix.
    """
    print(f"\n--- Generating SVD Recommendations (Latent Factors={k_factors}) ---")
    
    inv_user_map = {v: k for k, v in user_map.items()}
    all_recs = []
    
    # For metadata lookup
    item_map_dict = df_items_map.set_index('item_id').to_dict('index')
    
    # Inverse Item Map needed? 
    # Usually item_map in build_matrix was item_id -> col_idx
    # We need col_idx -> item_id
    # We assume 'item_map' passed to this function is the dict from build_matrix? 
    # Wait, we need to pass that map explicitly or recreate inverse here.
    # Let's assume we pass inv_item_map for efficiency or re-generate it.
    # For safety, let's pass the interaction df to re-derive consistent mapping or return maps from build fn.
    # We will assume we have 'item_id_list' correpsonding to columns.
    pass 

def run_svd_pipeline(R_centered, user_means, user_map, item_map, df_items_map, target_users=None, k=20):
    """
    Wrapper.
    """
    # 1. Prepare
    R_ready = prepare_matrix_for_svd(R_centered)
    
    # 2. Decompose
    u, sigma, vt = apply_truncated_svd(R_ready, k=k)
    
    # 3. Reconstruct
    pred_matrix = reconstruct_predictions(u, sigma, vt, user_means)
    
    # 4. Recommend
    # Inverse maps
    inv_user_map = {v: k for k, v in user_map.items()}
    inv_item_map = {v: k for k, v in item_map.items()}
    
    if target_users is None:
        # Default to first 5 users in map
        target_users = list(user_map.keys())[:5]
        
    all_recs = []
    
    for uid in target_users:
        if uid not in user_map:
            continue
        
        u_idx = user_map[uid]
        user_preds = pred_matrix[u_idx, :]
        
        # Identify already rated? Standard practice is to exclude them.
        # But SVD fills them with 'denoised' values. 
        # We usually want to recommend NEW items.
        # We need original R to check what was rated.
        # Since we only passed R_centered, we can check non-zeros in R_centered? 
        # R_centered has 0 for unrated, but also potentially 0 for rated-as-average.
        # Better to assume we recommend top scored items regardless or pass original R.
        # We will just rank top items for now.
        
        # Sort indices
        top_indices = np.argsort(user_preds)[::-1][:20]
        
        rank = 1
        for i_idx in top_indices:
            score = user_preds[i_idx]
            item_id = inv_item_map[i_idx]
            
            # Meta
            title = "Unknown"
            item_row = df_items_map[df_items_map['item_id'] == item_id]
            if not item_row.empty:
                title = item_row.iloc[0]['title']
            
            all_recs.append({
                'User': uid,
                'Rank': rank,
                'Item_ID': item_id,
                'Predicted_Rating': round(score, 4),
                'Title': title,
                'Method': f'SVD (k={k})'
            })
            rank += 1
            
    # Save
    df_out = pd.DataFrame(all_recs)
    save_path = os.path.join(RESULTS_DIR, "svd_recommendations.csv")
    df_out.to_csv(save_path, index=False)
    print(f"\nSVD Recommendations saved to {save_path}")
    print(df_out.head(10))
    return df_out

### Memory Optimization for SVD

**Critical Note**: Reconstructing the full dense matrix $(N_{users} \times N_{items})$ allows for global ranking but consumes massive memory (approx 2.8TB for this dataset), leading to `MemoryError`.

To resolve this, we implement **On-Demand Prediction**:
Instead of `np.dot(U, Sigma, Vt)`, we compute predictions only for the specific **target users** one by one.
Prediction for user $u$: $r_u = U_u \cdot \Sigma \cdot V^T + \mu_u$

In [ ]:
def predict_for_user_svd(u_idx, u_matrix, sigma_matrix, vt_matrix, user_mean):
    """
    Computes dense prediction vector for a single user to save memory.
    Result shape: (n_items, )
    """
    # Get User's latent vector: (1, k)
    user_latent = u_matrix[u_idx, :].reshape(1, -1)
    
    # Compute: User_Latent * Sigma * Vt
    # (1, k) * (k, k) -> (1, k)
    w = np.dot(user_latent, sigma_matrix)
    
    # (1, k) * (k, n_items) -> (1, n_items)
    user_preds = np.dot(w, vt_matrix).flatten()
    
    # Add Mean
    user_preds += user_mean
    
    # Clip
    return np.clip(user_preds, 1.0, 5.0)

In [ ]:
def generate_svd_recommendations_efficient(u_matrix, sigma_matrix, vt_matrix, user_means, user_map, item_map, df_items_map, target_users, k_factors):
    """
    Generates recommendations row-by-row to avoid OOM.
    """
    print(f"\n--- Generating Efficient SVD Recommendations (k={k_factors}) ---")
    
    inv_item_map = {v: k for k, v in item_map.items()}
    all_recs = []
    
    # Pre-fetch item titles for speed
    item_titles = pd.Series(df_items_map.title.values, index=df_items_map.item_id).to_dict()
    
    for uid in target_users:
        if uid not in user_map:
            continue
            
        u_idx = user_map[uid]
        mean_rating = user_means[u_idx]
        
        # Compute ONLY this user's predictions
        preds = predict_for_user_svd(u_idx, u_matrix, sigma_matrix, vt_matrix, mean_rating)
        
        # Sort top 20
        # argsort is fast on 1D array
        top_indices = np.argsort(preds)[::-1][:20]
        
        rank = 1
        for i_idx in top_indices:
            score = preds[i_idx]
            item_id = inv_item_map.get(i_idx, "Unknown")
            title = item_titles.get(item_id, "Unknown")
            
            all_recs.append({
                'User': uid,
                'Rank': rank,
                'Item_ID': item_id,
                'Predicted_Rating': round(score, 4),
                'Title': title,
                'Method': f'SVD (k={k_factors})'
            })
            rank += 1
            
    # Save
    df_out = pd.DataFrame(all_recs)
    save_path = os.path.join(RESULTS_DIR, "svd_recommendations.csv")
    df_out.to_csv(save_path, index=False)
    print(f"Recommendations saved to {save_path}")
    print(df_out.head(10))
    return df_out

In [ ]:
def run_svd_pipeline_efficient(R_centered, user_means, user_map, item_map, df_items_map, target_users=None, k=20):
    """
    Memory-Efficient Wrapper.
    """
    # 1. Prepare
    R_ready = prepare_matrix_for_svd(R_centered)
    
    # 2. Decompose
    u, sigma, vt = apply_truncated_svd(R_ready, k=k)
    
    # 3. Recommend (Skip full reconstruction)
    if target_users is None:
        target_users = list(user_map.keys())[:5]
        
    svd_recs = generate_svd_recommendations_efficient(
        u, sigma, vt, user_means, user_map, item_map, df_items_map, target_users, k
    )
    
    return svd_recs